<a href="https://colab.research.google.com/github/anutashakina/linal_lab/blob/main/Linal_chernovik.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # временное
import random
import math

def generate_matrix(rows, cols, min_val, max_val):
    return [[random.randint(min_val, max_val) for i in range(cols)] for j in range(rows)]

- [x] Метод Гаусса
- [x] Центрирование
- [x] Матрица ковариаций
- [x] Собственные значения
- [x] Собственные векторы
- [x] Доля объяснённой дисперсии
- [x] PCA
- [ ] Визуализация проекции на первые две главные компоненты
- [ ] Вычисление MSE
- [ ] Автоматический выбор числа главных компонент
- [ ] Обработка пропущенных значений
- [ ] Влияние шума на PCA
- [ ] PCA in real data
- [ ] Теоретическая часть

# Easy 1: Метод Гаусса

In [ ]:
# Основная функция
def gauss_method(A, b, tol):
    # Делаем расширенную матрицу (A|b)
    n = len(A)
    m = len(A[0]) if n > 0 else 0
    Ab = [[0.0 for _ in range(m + 1)] for _ in range(n)]

    for i in range(n):
        for j in range(m):
            Ab[i][j] = A[i][j]
        Ab[i][m] = b[i][0] if isinstance(b[0], list) else b[i]

    # Прямой ход Гаусса
    rank = 0
    for col in range(m):
        # Ищем строку с макс элементом в текущем столбце
        max_row = rank
        for row in range(rank + 1, n):
            if abs(Ab[row][col]) > abs(Ab[max_row][col]):
                max_row = row

        # Если максимальный элемент равен нулю, скипаем столбец
        if abs(Ab[max_row][col]) < tol:
            continue

        # Меняем строки местами
        Ab[rank], Ab[max_row] = Ab[max_row], Ab[rank]

        # Нормализация строки
        pivot = Ab[rank][col]
        for j in range(col, m + 1):
            Ab[rank][j] /= pivot

        # Исключение переменной из других строк
        for i in range(n):
            if i != rank and abs(Ab[i][col]) > tol:
                factor = Ab[i][col]
                for j in range(col, m + 1):
                    Ab[i][j] -= factor * Ab[rank][j]

        rank += 1
        if rank == n:
            break

    # Проверяем совместность
    for i in range(rank, n):
        if abs(Ab[i][m]) > tol:
            return []

    # Обратный ход Гаусса
    solutions = []
    free_vars = []

    # Определяем свободные переменные
    lead_vars = [-1] * m
    for i in range(rank):
        for j in range(m):
            if abs(Ab[i][j]) > tol:
                lead_vars[j] = i
                break

    for j in range(m):
        if lead_vars[j] == -1:
            free_vars.append(j)

    # Если свободных нет - единственное решение
    if not free_vars:
        solution = [[0.0] for _ in range(m)]
        for i in range(rank):
            for j in range(m):
                if abs(Ab[i][j]) > tol:
                    solution[j][0] = Ab[i][m]
                    break
        return [solution]

    # Если свободные есть - строим базис
    for free in free_vars:
        vec = [[0.0] for _ in range(m)]
        vec[free][0] = 1.0

        for i in range(rank):
            for j in range(m):
                if abs(Ab[i][j]) > tol:
                    sum_ax = 0.0
                    for k in range(j + 1, m):
                        sum_ax += Ab[i][k] * vec[k][0]
                    vec[j][0] = Ab[i][m] - sum_ax
                    break

        solutions.append(vec)

    return solutions

# Easy 2: Центрирование данных
$$ X_{centered} = X - X_{mean} $$

In [ ]:
def get_means(X):
  rows = len(X)
  cols = len(X[0])

  # средние по столбцам
  means = []
  for j in range(cols):
    col_sum = 0
    for i in range(rows):
      col_sum += X[i][j]
    means.append(col_sum/rows)

  return means

def center_data(X, means):
  X_centered = [
      [X[i][j] - means[j] for j in range(len(X[0]))] for i in range(len(X))
  ]
  return X_centered

In [ ]:
C = generate_matrix(5, 5, -100, 100)
print(C)
means = get_means(C)
print(center_data(C, means))

[[-22, -100, 92, 14, -59], [49, 2, 31, -63, 42], [-95, 85, 0, -52, -32], [-52, -33, -37, -46, 98], [8, 97, 37, 4, -36]]
[[0.3999999999999986, -110.2, 67.4, 42.6, -61.6], [71.4, -8.2, 6.399999999999999, -34.4, 39.4], [-72.6, 74.8, -24.6, -23.4, -34.6], [-29.6, -43.2, -61.6, -17.4, 95.4], [30.4, 86.8, 12.399999999999999, 32.6, -38.6]]


# Easy 3: матрица ковариаций
$$ C = \frac{1}{n-1}X^TX $$

In [ ]:
# транспонирование, умножение матрица*матрица и скаляр*матрица
def transpose_matrix(X):
  return [
        [X[i][j] for i in range(len(X))]
        for j in range(len(X[0]))
    ]

def scalar_multiply(scalar, matrix):
  return [[element * scalar for element in row] for row in matrix]

def matrix_multiply(A, B):
  m = len(A)
  p = len(B[0])
  result = [[0] * p for _ in range(m)]

  for i in range(m):
      for j in range(p):
          for k in range(len(B)):
              result[i][j] += A[i][k] * B[k][j]

  return result

In [ ]:
# вычисление матрицы ковариаций
def covariance_martix(X_centered):
  n = len(X_centered)
  X_T = transpose_matrix(X_centered)
  scalar = 1 / (n - 1)

  cov_matrix = scalar_multiply(scalar, matrix_multiply(X_T, X_centered))
  return cov_matrix

In [ ]:
C = generate_matrix(5, 5, -100, 100)
C_cov = covariance_martix(center_data(C))
print(C)
print(C_cov)

[[23, -16, 55, 55, -63], [-46, -8, -28, 10, 23], [-85, -77, -67, -36, -3], [95, 70, 56, 92, 18], [81, -31, -74, -84, 19]]
[[6132.800000000001, 2881.8, 2090.7000000000003, 1324.4499999999998, 269.6500000000001], [2881.8, 2835.3, 2499.45, 2828.7000000000003, 412.90000000000003], [2090.7000000000003, 2499.45, 4059.2999999999997, 4238.549999999999, -1093.8999999999996], [1324.4499999999998, 2828.7000000000003, 4238.549999999999, 4916.799999999999, -755.65], [269.6500000000001, 412.90000000000003, -1093.8999999999996, -755.65, 1296.2]]


# Normal 1: Собственные значения

In [ ]:
# поиск корня бисекцией
def root_search(f, a, b, tol=1e-6):
    fa, fb = f(a), f(b)
    if fa * fb > 0:
        return None

    while (b-a) > tol:
        c = (a + b) / 2
        fc = f(c)
        if abs(fc) < tol:
            return c
        if fa * fc < 0:
            b, fb = c, fc
        else:
            a, fa = c, fc
    return (a + b) / 2

# поиск экстремума бисекцией
def extremum_search(f, a0, b0, epsilon, delta=1e-10):
    a = a0
    b = b0
    ans = (a + b) / 2

    while abs(b - a) > epsilon:
        yk = (a + b - delta) / 2
        zk = (a + b + delta) / 2

        if f(yk) <= f(zk):
            b = zk
        else:
            a = yk

        ans = (a + b) / 2
    return ans

In [ ]:
# функция определителя
def determinant(mat):
    n = len(mat)
    if n == 1:
        return mat[0][0]
    if n == 2:
        return mat[0][0]*mat[1][1] - mat[0][1]*mat[1][0]

    det = 0
    for col in range(n):
        minor = [row[:col] + row[col+1:] for row in mat[1:]]
        det += (-1)**col * mat[0][col] * determinant(minor)

    return det

In [ ]:
# основная функция. поиск собственных значений
def bisection_eigenvalues(C, tol=1e-5):
    n = len(C)

    # обычный характеристический полином
    def characteristic_poly(lam):
        mat = [[C[i][j] - (lam if i == j else 0) for j in range(n)] for i in range(n)]
        #return determinant(mat)
        # чтоб оперативней считалось, потом закомментить
        return np.linalg.det(mat)

    # модуль полинома для корней чётной кратности
    def poly_abs(lam):
        mat = [[C[i][j] - (lam if i == j else 0) for j in range(n)] for i in range(n)]
        #return abs(determinant(mat))
        # чтоб оперативней считалось, потом закомментить
        return abs(np.linalg.det(mat))

    # Вычисляем интервал поиска (см теорему Гершгорина)
    Gershgorin_intervals = []
    for i in range(n):
        radius = sum(abs(C[i][j]) for j in range(n) if j != i)
        center = C[i][i]
        Gershgorin_intervals.append((center - radius, center + radius))

    lower = min(interval[0] for interval in Gershgorin_intervals)
    upper = max(interval[1] for interval in Gershgorin_intervals)

    # Расширяем интервал, ну чтобы понадёжнее
    lower -= 3
    upper += 3
    count = math.ceil(0.5*upper - 0.5*lower)

    # непосредственно поиск корней
    def find_roots(f, fabs, a, b, tol, count):
        roots = []
        # дробим [a,b] на мелкие подотрезки
        step = (b - a) / count

        x_prev = a
        f_prev = f(x_prev)

        # пробегаем по каждому маленькому отрезку
        for i in range(1, count+1):
            x = a + i * step
            fx = f(x)

            # если попали в корень
            if abs(fx) < tol:
                roots.append(x)
            # если функция меняет знак, ищем корень
            # если не меняет, смотрим экстремум для функции-модуля.
            if f_prev * fx < 0:
                roots.append(root_search(f, x_prev, x, tol))
            else:
                exst = extremum_search(fabs, x_prev, x, tol)
                # если при поиске экстремума «скатились» близко к нулю, то всё ок
                if abs(f(exst)) < 0.0001:
                  roots.append(exst)

            x_prev = x
            f_prev = fx

        # убираем дубликаты (близкие корни)
        unique_roots = [roots[0]]
        for i in range(1,len(sorted(roots))):
            if not unique_roots or (roots[i] - roots[i-1] > 2*step):
                unique_roots.append(roots[i])

        return unique_roots

    eigenvalues = find_roots(characteristic_poly, poly_abs, lower, upper, tol, count)
    return sorted(eigenvalues)[::-1]

In [ ]:
# Пример использования
C = generate_matrix(10, 10, -100, 100)
C_cov = covariance_martix(center_data(C))
tol = 1e-6
print("Найденные собственные значения (вещественные):", bisection_eigenvalues(C_cov, tol))
print("Правильные собственные значения (+ комплексные):", np.linalg.eigvals(C_cov))

Найденные собственные значения (вещественные): [9227.366191429148, 7367.136653184566, 6553.423427497663, 4289.309408212141, 2787.027904053094, 1462.9687426778535, 402.0354219261742, 137.44614762076412, 48.286103585758596, -8.967490345646381e-08]
Правильные собственные значения (+ комплексные): [ 9.22736619e+03  7.36713665e+03  6.55342343e+03  4.28930941e+03
  2.78702790e+03  1.46296874e+03  4.02035422e+02  1.37446148e+02
 -2.40999980e-13  4.82861039e+01]


# Normal 2: Собственные векторы

In [ ]:
def normalize_vector(vector):
    norm = math.sqrt(sum(x[0] ** 2 for x in vector))
    return [[x[0] / norm] for x in vector]

def find_eigenvectors(C, eigenvalues, tol=1e-2):
  n = len(C)
  vectors_list = []

  for lam in eigenvalues:
    C_lam = [[C[i][j] - (lam if i == j else 0) for j in range(n)] for i in range(n)]
    vectors_lam = gauss_method(C_lam, [[0]]*n, tol)
    vectors_list += vectors_lam

  normalize_list = []
  for vector in vectors_list:
    normalize_list.append(normalize_vector(vector))

  return normalize_list

In [ ]:
C = [[-3, 18, 92, 50, 80], [67, -86, -25, -15, 94], [-65, 6, -66, 18, 67], [22, -1, -39, -41, 87], [-85, -3, 26, 2, 22]]
covar = covariance_martix(center_data(C))

print('результат обычный')
vals = bisection_eigenvalues(covar)
vects = find_eigenvectors(covar, vals, 1e-2)
print(vals)
for v in vects:
  print(v)

print('\nрезультат нампая:')
npvals, npvects = np.linalg.eig(covar)
print(npvals)
print(npvects)

результат обычный
[6093.239925590242, 3978.734686585489, 887.5278085873667, 563.8975791470539, -6.388686481294086e-07]
[[0.6756005803453531], [-0.4175990407830971], [-0.45189063593575546], [-0.2994169598343279], [0.27444349926935085]]
[[0.5194698484242519], [-0.07784347808905089], [0.8061063255347838], [0.21620209823200293], [0.16595395189621923]]
[[0.2206068322334603], [0.8018904775951237], [-0.20521201864870794], [0.1433662277826908], [0.49561924866541096]]
[[-0.09578979790129942], [-0.3428685134443979], [-0.2535171307369759], [0.8663255423416735], [0.24181525223970768]]
[[-0.4646201386322448], [-0.24282561247418047], [0.1990088853618068], [-0.3041912480599674], [0.770082461074568]]

результат нампая:
[6.09323993e+03 3.97873468e+03 1.16635341e-13 8.87527809e+02
 5.63897580e+02]
[[-0.67560058  0.51946985  0.46462014  0.22060683 -0.0957898 ]
 [ 0.41759904 -0.07784348  0.24282561  0.80189048 -0.34286851]
 [ 0.45189064  0.80610633 -0.19900889 -0.20521202 -0.25351713]
 [ 0.29941696  0.216

# Normal 3: Доля дисперсии

In [ ]:
def explained_variance_ratio(eigenvalues, k):
  return sum(eigenvalues[:k]) / sum(eigenvalues)

In [ ]:
C = [[-3, 18, 9, 50, 80], [7, -86, -25, -15, 94], [-65, 6, -66, 18, 67], [22, -1, -39, -41, 87], [-85, -3, 26, 2, 22]]
covar = covariance_martix(center_data(C))

vals = bisection_eigenvalues(covar)
print(vals)
print(explained_variance_ratio(vals, 1), explained_variance_ratio(vals, 3), explained_variance_ratio(vals, 5))

[3726.2397348370037, 1519.927697036278, 1311.9728830331337, 751.4596884947401, 3.716558865184995e-06]
0.5097734118431353 0.8971955111798776 1.0


# Hard 1: алгоритм PCA

In [ ]:
def project_data(X, eigenvectors, k):
  vector_matrix = []

  for i in range(len(X)):
    row = []
    for v in eigenvectors[:k]:
        row.append(v[i][0])
    vector_matrix.append(row)

  project_X = matrix_multiply(X, vector_matrix)
  return (project_X, vector_matrix)

def pca(X, k):
  # центрирование данных и матрица ковариаций
  means_X = get_means(X)
  centered_X = center_data(X, means_X)
  covariance_X = covariance_martix(centered_X)

  # поиск собственных значений и векторов
  eigenvalues = bisection_eigenvalues(covariance_X)
  eigenvectors = find_eigenvectors(covariance_X, eigenvalues)

  # проекция данных на главные компоненты
  projected_X = project_data(centered_X, eigenvectors, k)[0]
  dispersion_result = explained_variance_ratio(eigenvalues, k)

  return (projected_X, dispersion_result)

In [ ]:
C = [[-3, 18, 92, 50, 80], [67, -86, -25, -15, 94], [-65, 60, -66, 18, 67], [22, -1, -39, -41, 87], [-85, -3, 26, 2, 22]]
res = pca(C,3)[0]

for row in res:
  print(row)

[-32.95151971152623, 97.76296747184719, 33.28401724781072]
[119.04337271336402, 7.896886711963553, -14.995757099685436]
[-63.36476846698973, -77.02366914462185, 26.844414355315926]
[47.56194048124183, -39.13442833317077, 11.969039564062477]
[-70.28902501608991, 10.498243293981877, -57.10171406750369]


In [ ]:
from sklearn.decomposition import PCA
pca2 = PCA(n_components=3)
data_pca = pca2.fit_transform(C)
print(data_pca)

[[-32.95151973  97.76296747  33.28401724]
 [119.0433727    7.89688672 -14.99575711]
 [-63.36476843 -77.02366915  26.8444144 ]
 [ 47.56194051 -39.13442833  11.96903953]
 [-70.28902505  10.49824329 -57.10171405]]
